In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../input/heartdisease/heart_disease.csv')


data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [3]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [4]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

model = XGBClassifier(booster='gbtree',random_state=2, objective='binary:logistic')

In [5]:
scores = cross_val_score(model, X, y, cv=5)
print('Accuracy:',np.round(scores, 2))
print('Accuracy mean: %.2f' % scores.mean())

Accuracy: [0.84 0.85 0.82 0.8  0.77]
Accuracy mean: 0.81


# Hyperparameter Tuning

In [6]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=2, shuffle=True)

Now that we have a method for obtaining consistent folds, it's time to score an official baseline model using cv=kfold inside cross_val_score

In [7]:
scores = cross_val_score(model, X, y, cv=kfold)
print('Accuracy:',np.round(scores,2))
print('Accuracy mean: %.2f' % scores.mean())

Accuracy: [0.72 0.82 0.75 0.8  0.82]
Accuracy mean: 0.78


In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def grid_search(params, random=False):
    xgb = XGBClassifier(booster='gbtree', random_state=2, objective='binary:logistic')
    
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_jobs=-1, n_iter=20)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    
    grid.fit(X, y)
    best_params = grid.best_params_
    print('Best params:',best_params)
    
    best_score = grid.best_score_
    print('Training score: %.5f' % best_score)
       

In [9]:
grid_search(params={'n_estimators':[100,200,400.800]})

Best params: {'n_estimators': 100}
Training score: 0.78235


In [10]:
grid_search(params={'learning_rate': [0.01,0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best params: {'learning_rate': 0.05}
Training score: 0.79585


In [11]:
grid_search(params={'max_depth':[2,3,5,6,8]})

Best params: {'max_depth': 2}
Training score: 0.79902


In [12]:
grid_search(params={'gamma':[0, 0.1, 0.5, 1, 2, 5]})

Best params: {'gamma': 0.5}
Training score: 0.79574


In [13]:
grid_search(params={'min_child_weight':[1,2,3,4,5]})

Best params: {'min_child_weight': 5}
Training score: 0.81219


In [14]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'subsample': 0.8}
Training score: 0.79579


In [15]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'colsample_bytree': 0.5}
Training score: 0.80219


In [16]:
grid_search(params = {'colsample_bylevel':[0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'colsample_bylevel': 0.8}
Training score: 0.80240


In [17]:
grid_search(params = {'colsample_bynode': [ 0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'colsample_bynode': 0.8}
Training score: 0.80240


# Early stopping round, eval_metric, eval_set

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)

eval_metric = 'error'
eval_set = [(X_test, y_test)]

model.fit(X_train,y_train, eval_metric=eval_metric, eval_set=eval_set)

y_preds = model.predict(X_test)

accuracy = accuracy_score(y_test, y_preds)
print('Accuracy Score: %.2f%%' % (accuracy * 100.0))

[0]	validation_0-error:0.15790
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15790
[12]	validation_0-error:0.15790
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15790
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15790
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

# early stopping rounds

In [19]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)
eval_set = [(X_test, y_test)]
eval_metric = 'error'
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, early_stopping_rounds=10, verbose=True)
y_preds = model.predict(X_test)
accuracy = accuracy_score(y_preds, y_test)
print('Accuracy: %0.2f%%' % (accuracy * 100.0))

[0]	validation_0-error:0.15790
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15790
Stopping. Best iteration:
[1]	validation_0-error:0.10526

Accuracy: 89.47%


In [20]:
model = XGBClassifier(random_state = 2, n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric = 'error'
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, early_stopping_rounds=100)
y_preds = model.predict(X_test)
accuracy = accuracy_score(y_preds, y_test)
print('Accuracy: %0.2f%%' % (accuracy * 100.0))

[0]	validation_0-error:0.15790
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15790
[12]	validation_0-error:0.15790
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15790
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15790
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation

# Combining Hyperparameters

In [21]:
grid_search(params = {'n_estimators':[2,25,50,75,100]})

Best params: {'n_estimators': 50}
Training score: 0.78907


In [22]:
grid_search(params = {'max_depth':[1,2,3,4,5,6,7,8], 'n_estimators':[50]})

Best params: {'max_depth': 1, 'n_estimators': 50}
Training score: 0.83869


In [23]:
grid_search(params = {'learning_rate': [0.01, 0.05, 0.1, 0.2,0.3,0.4,0.5],
                     'max_depth': [1],
                     'n_estimators':[50]})

Best params: {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
Training score: 0.83869


In [24]:
grid_search(params = {'min_child_weight':[1,2,3,4,5], 
                         'max_depth':[1],
                        'n_estimators':[50]})

Best params: {'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 50}
Training score: 0.83869


In [27]:
grid_search(params = {'subsample':[0.5,0.6,0.7,0.8,0.9],
                     'max_depth':[1],
                     'n_estimators':[50]})

Best params: {'max_depth': 1, 'n_estimators': 50, 'subsample': 0.5}
Training score: 0.83514


In [28]:
# comprehensive grid search of values used

grid_search(params = {'subsample':[0.5,0.6,0.7,0.8,0.9],
                     'min_child_weight':[1,2,3,4,5],
                     'learning_rate': [0.1, 0.2,0.3,0.4,0.5],
                     'max_depth':[1,2,3,4,5],
                     'n_estimators':[2]})

Best params: {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
Training score: 0.81224


In [29]:
grid_search(params = {'subsample':[0.5,0.6,0.7,0.8,0.9,1],
                     'min_child_weight':[1,2,3,4,5],
                     'learning_rate': [0.1, 0.2,0.3,0.4,0.5],
                     'max_depth':[1,2,3,4,5,None],
                     'n_estimators':[2,25,50,75,100]},
                       random=True)

Best params: {'subsample': 0.6, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 4, 'learning_rate': 0.1}
Training score: 0.82208


In [30]:
# let's try colsample_bytree, colsample_bylevel, and colsample_bynode

grid_search(params = {'colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1],
                     'max_depth':[1],
                     'n_estimators':[50]})

Best params: {'colsample_bytree': 0.5, 'max_depth': 1, 'n_estimators': 50}
Training score: 0.84197


In [31]:
grid_search(params = {'colsample_bylevel':[0.5,0.6,0.7,0.8,0.9,1],
                     'max_depth':[1],
                     'n_estimators':[50]})

Best params: {'colsample_bylevel': 0.5, 'max_depth': 1, 'n_estimators': 50}
Training score: 0.84197


In [32]:
grid_search(params = {'colsample_bynode':[0.5,0.6,0.7,0.8,0.9,1],
                      'colsample_bylevel':[0.5,0.6,0.7,0.8,0.9,1],
                      'colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1],
                     'max_depth':[1],
                     'n_estimators':[50]})

Best params: {'colsample_bylevel': 0.5, 'colsample_bynode': 0.7, 'colsample_bytree': 0.5, 'max_depth': 1, 'n_estimators': 50}
Training score: 0.85169


In [34]:
grid_search(params = {'gamma':[0,0.01,0.05,0.1,0.5,1,2,3],
                       'colsample_bylevel': [0.5], 
                       'colsample_bynode': [0.7], 
                       'colsample_bytree': [0.5], 
                       'max_depth': [1], 
                       'n_estimators': [50]})

ValueError: Parameter grid for parameter (colsample_bylevel) needs to be a list or numpy array, but got (<class 'float'>). Single values need to be wrapped in a list with one element.